<a href="https://colab.research.google.com/github/monsterspy/Results-Screener/blob/main/Results_Screener.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nsepython
import nsepy
import pandas as pd
import numpy as np
from datetime import timedelta, date
DAYS_AHEAD = 14
DAYS_BEHIND = 7

In [2]:
def get_stocks_with_upcoming_results():
    results = nsepython.nse_events()
    results['date'] = pd.to_datetime(results['date'])
    results = results[(results['purpose'] == "Financial Results") & (results['date'] <= pd.to_datetime(date.today() + timedelta(days=DAYS_AHEAD)))].drop_duplicates(subset=["symbol"])
    return results

In [3]:
results = get_stocks_with_upcoming_results()

In [4]:
def get_relative_change(df):
    close = df["Close"].values
    change_pct = (close[len(close)-1]-close[0])/close[0]*100
    return round(change_pct, 2)


In [5]:
def get_no_higher_highs(df):
    cnt = 0
    close = df["Close"].values
    for idx in range(1, len(close)):
        if close[idx] >= close[idx-1]:
            cnt+=1
    return cnt

In [6]:
def is_increasing(df):
    delv = df["%Deliverble"].values
    n = len(delv)
    if (delv[n-1] >= delv[n-2]) and (delv[n-2] >= delv[n-3]):
        return True
    return False

In [7]:
start_date = date.today() - timedelta(days=DAYS_BEHIND)
ticker_info = []

for row in results.iterrows():
    symbol = row[1]['symbol']
    date = row[1]['date'].date()

    try:
        data = nsepy.get_history(symbol=symbol, start=start_date, end=date.today())
        if len(data) == 0:
            continue
        relative_change = get_relative_change(data)
        no_higher_highs = get_no_higher_highs(data)
        is_delivery_upside = is_increasing(data)

        ticker_info.append([symbol, date, relative_change, no_higher_highs, is_delivery_upside])
    except Exception as e:
        print(e)
    

    

index -2 is out of bounds for axis 0 with size 1


In [8]:
tickers_df = pd.DataFrame(ticker_info)
tickers_df.columns = ["symbol", "results_date", "relative_change_7D", "higher_highs_count", "is_delivery_increasing"]
tickers_df = tickers_df.sort_values(by=["relative_change_7D"], ascending=False).reset_index(drop=True)

In [9]:
tickers_df.head()

,symbol,results_date,relative_change_7D,higher_highs_count,is_delivery_increasing
0,SURANASOL,2021-10-22,32.93,4,False
1,BORORENEW,2021-10-21,30.41,3,False
2,TATAPOWER,2021-10-28,26.13,3,False
3,DMART,2021-10-16,18.68,4,False
4,INDHOTEL,2021-10-21,17.69,4,False


In [10]:
tickers_df.to_csv("sample_results.csv", index=False)